# Imports

In [7]:
import numpy as np
import pandas as pd
import string
import collections
import nagisa

# Read in data

In [2]:
wiki_df = pd.read_pickle('wiki') 
wiki_df.head()

,article,views,text
0,メインページ,362562853,ようこそ\nウィキペディア - ウィキペディア日本語版 - 百科事典目次\n検索資料・ポータ...
1,星野源,10190763,星野 源（ほしの みなもと、1981年1月28日 - ）は、日本の音楽家、俳優、文筆家。埼玉...
2,真田信繁,9602104,真田 信繁（さなだ のぶしげ）は、安土桃山時代から江戸時代初期にかけての武将、大名。真田昌幸...
3,高橋一生,8571666,高橋 一生（たかはし いっせい、英字表記：Issey Takahashi、1980年12月9...
4,君の名は。,7788879,『君の名は。』（きみのなは、英: Your Name.）は、2016年に公開された新海誠監督...


In [5]:
# Create a list of stop words
stop = ['あそこ','あっ','あの','あのかた','あの人','あり','あります','ある','あれ','い','いう','います','いる','う','うち','え','お','および','おり','おります','か','かつて','から','が','き','ここ','こちら','こと','この','これ','これら','さ','さらに','し','しかし','する','ず','せ','せる','そこ','そして','その','その他','その後','それ','それぞれ','それで','た','ただし','たち','ため','たり','だ','だっ','だれ','つ','て','で','でき','できる','です','では','でも','と','という','といった','とき','ところ','として','とともに','とも','と共に','どこ','どの','な','ない','なお','なかっ','ながら','なく','なっ','など','なに','なら','なり','なる','なん','に','において','における','について','にて','によって','により','による','に対して','に対する','に関する','の','ので','のみ','は','ば','へ','ほか','ほとんど','ほど','ます','また','または','まで','も','もの','ものの','や','よう','より','ら','られ','られる','れ','れる','を','ん','何','及び','彼','彼女','我々','特に','私','私達','貴方','貴方方''ようこそ','ウィキペディア','ウィキペディア日本語版','百科事典目次','検索資料','空白','補助記号','）', '（', '。', '、', '『', '』', '・', '！','：', '／', '＋', '→', '「', '」', '\n', ' ', '-', '/', '\\']
#stop += list(string.punctuation)
stop += list(string.printable)

In [6]:
# Extract data from dataframe
data = np.array(wiki_df['text'])

# Data Cleaning & Summarization Options

## Data cleaning - remove all stop characters

In [ ]:
new_data = []
for text in data:
    new_text = ""
    for character in text:
        if character not in stop:
            new_text += character
    new_data.append(new_text)

## Text features using Japanese grammar


は   wa   topic     (after)
が   ga   nontopic  (after)
を   o    object    (after)
する suru verb      (after)
る   ru   verb      (after)
な   na   adjective (after)
い   i    adjective (after)
の   no   adj or loc(before)

distance of each of the above

start of a sentence
end of a sentence

In [ ]:
article_num = 0
features = []
for article in data[0:1000]:
    sentence_num = 0
    for sentence in re.split('\n|。', article):
        cleaned = re.sub('['+'|'.join(list(string.printable))+']','',sentence)
        for i in range(len(cleaned)):
            wa = cleaned[i:].find('は')
            ga = cleaned[i:].find('が')
            o = cleaned[i:].find('を')
            suru = cleaned[i:].find('する')
            ru = cleaned[i:].find('る')
            na = cleaned[i:].find('な')
            ii = cleaned[i:].find('い')
            no = cleaned[(len(cleaned)-i)::-1].find('の')
            if i==0: 
                start_of_sentence = 1
            else:
                start_of_sentence = 0
            if i==len(cleaned):
                end_of_sentence = 1
            else:
                end_of_sentence = 0
            char_data = [cleaned[i], article_num, sentence_num, wa, ga, o, suru, ru, na, ii, no, start_of_sentence, end_of_sentence]
            features.append(char_data)
        sentence_num += 1
    article_num += 1
df = pd.DataFrame(features)
df.columns = ['character','articleNumber','sentenceNumber','wa','ga','o','suru','ru','na','ii','no','startSentence','endSentence']

## Find most commonly used character combinations

In [ ]:
word_combo_dict1 = collections.Counter()
word_combo_dict2 = collections.Counter()
word_combo_dict3 = collections.Counter()

for article in data[0:1000]:
    for sentence in re.split('\n|。', article):
        cleaned = re.sub('['+'|'.join(list(string.printable))+']','',sentence)
        for i in range(len(cleaned)):
            word_combo_dict1[cleaned[i]] += 1
            if i < (len(cleaned)-2):
                word_combo_dict2[cleaned[i:i+2]] += 1
            if i < (len(cleaned)-3):
                word_combo_dict3[cleaned[i:i+3]] += 1

In [ ]:
word_combo_dict1.most_common(10)

In [ ]:
word_combo_dict2.most_common(10)

In [ ]:
word_combo_dict3.most_common(10)

## Generate nagisa labeled data

In [ ]:
labeled_data = []
# use first 150 articles only due to data size
for doc_num in range(150):
    cur_text = nagisa.tagging(data[doc_num])
    words = cur_text.words
    postags = cur_text.postags
    char_offset = 0
    for i in range(len(words)):
        labeled_data.append([doc_num, char_offset, words[i], postags[i]])
    print("finished " + str(doc_num))
labeled_df = pd.DataFrame(labeled_data)
labeled_df.to_pickle('labeledData.pkl')